## Bonus Question - Connected Components on MapReduce

### MapReduce is ideal for network analysis as it enables parallel processing of large graph datasets, making it scalable and efficient. By breaking tasks into map and reduce steps, it allows for distributed analysis of networks, which is essential for handling large-scale graph problems like connected components.

### 1. In this task, you are required to use PySpark and the MapReduce paradigm to identify the connected components in a flight network graph. The focus should be on airports rather than cities. As you know, a connected component refers to a group of airports where every pair of airports within the group is connected either directly or indirectly.

### The function takes the following inputs:

### 1. Flight network
### 2. A starting date
### 3. An end date

### The function outputs:
### 1. The number of the connected components during that period
### 2. The size of each connectd componenet
### 3. The airports within the largest connected component identified.

### **Note**: For this task, you should check if there is a flight between two airports during that period. Note: You are not allowed to use pre-existing packages or functions in PySpark; instead, you must implement the algorithm from scratch using the MapReduce paradigm.

#### Mounting Google Drive and Installing Necessary Packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install PySpark and all the necessary packages
! pip install pyngrok gdown  pyspark  yellowbrick graphframes

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark import SparkContext, StorageLevel
from pyspark.sql.functions import col, explode, collect_list, min, to_date as spark_min, struct, size, row_number, to_date
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from graphframes import GraphFrame
import time
import json
import tempfile
import os

In [4]:
# Get the active SparkContext instance
sc = SparkContext.getOrCreate()

# Stop SparkContext if you want to create a new one
sc.stop()

# Initialize Spark Session with configurations
print("Initializing Spark Session...")
spark = SparkSession.builder \
    .appName("Connected Components Analysis") \
    .config("spark.driver.memory", "15g") \
    .config("spark.executor.memory", "15g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()
print("... Spark Session Created")
sc = spark.sparkContext

Initializing Spark Session...
... Spark Session Created


#### Loading and Preparing the Flight Network Dataset

We perform the following operations to load and prepare the flight network dataset for analysis:

1. **Loading the Dataset with Pandas**;

2. **Converting Pandas DataFrame to Spark DataFrame**;

3. **Displaying the Schema of the Spark DataFrame**.



In [5]:
# Load the dataset as a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Airports2.csv')
flight_network_df = spark.createDataFrame(df)

# Display the schema to confirm
flight_network_df.printSchema()

root
 |-- Origin_airport: string (nullable = true)
 |-- Destination_airport: string (nullable = true)
 |-- Origin_city: string (nullable = true)
 |-- Destination_city: string (nullable = true)
 |-- Passengers: long (nullable = true)
 |-- Seats: long (nullable = true)
 |-- Flights: long (nullable = true)
 |-- Distance: long (nullable = true)
 |-- Fly_date: string (nullable = true)
 |-- Origin_population: long (nullable = true)
 |-- Destination_population: long (nullable = true)
 |-- Org_airport_lat: double (nullable = true)
 |-- Org_airport_long: double (nullable = true)
 |-- Dest_airport_lat: double (nullable = true)
 |-- Dest_airport_long: double (nullable = true)



We define two crucial functions, `propagate_labels` and `identify_connected_components_rdd`, which work together to identify connected components within a flight network dataset using Spark's Resilient Distributed Datasets (RDDs). This approach leverages low-level Spark transformations and actions for efficient graph processing.
- The `propagate_labels` function is a helper function designed to facilitate the iterative process of label propagation, a key step in identifying connected components within a graph. By propagating the smallest label across connected nodes, the function ensures that all nodes within the same connected component eventually share the same label.
- The `identify_connected_components_rdd` function is designed to identify connected components within a flight network dataset over a specified date range using Spark's RDD API. Connected components represent groups of airports where each airport is reachable from any other airport within the same group, indicating a fully interconnected sub-network.


In [11]:
import builtins
from pyspark.sql.functions import to_date, col
from pyspark import StorageLevel
import time

def propagate_labels(labels_rdd, graph_rdd):
    """
    Propagates labels through the graph to identify connected components.

    Parameters:
    - labels_rdd (RDD of tuples): Current labels as (airport_id, label_id).
    - graph_rdd (RDD of tuples): Graph edges as (airport_id, list of neighbor_ids).

    Returns:
    - new_labels (RDD of tuples): Updated labels after propagation as (airport_id, new_label_id).
    """
    # Join graph with current labels to get the label of each airport's neighbors
    joined = graph_rdd.join(labels_rdd)

    # Create messages to propagate the smallest label to each neighbor
    messages = joined.flatMap(lambda x: [(neighbor, x[1][1]) for neighbor in x[1][0]])

    # Combine incoming messages with existing labels and choose the minimum label
    new_labels = messages.union(labels_rdd).reduceByKey(lambda a, b: builtins.min(a, b))

    return new_labels

def identify_connected_components_rdd(flight_network_df, start_date, end_date):
    """
    Identifies the connected components in the flight network between start_date and end_date.

    Parameters:
    - flight_network_df (DataFrame): Spark DataFrame containing flight data with columns
                                      'Origin_airport', 'Destination_airport', 'Fly_date'.
    - start_date (str): Start date of the period in 'YYYY-MM-DD' format.
    - end_date (str): End date of the period in 'YYYY-MM-DD' format.

    Returns:
    - num_connected_components (int): Number of connected components.
    - component_sizes_sorted (list of tuples): List of component sizes sorted in descending order.
    - largest_component_airports (list): List of airports in the largest connected component.
    - execution_time (float): Time taken to execute the function in seconds.
    """

    # Filter flights within the specified date range and drop rows with null airports
    filtered_df = flight_network_df.filter(
        (col("Fly_date") >= start_date) & (col("Fly_date") <= end_date)
    ).na.drop(subset=["Origin_airport", "Destination_airport"])

    # Create edges by selecting origin and destination airports and ensuring uniqueness
    edges = filtered_df.select(
        col("Origin_airport").alias("src"),
        col("Destination_airport").alias("dst")
    ).rdd.map(lambda row: (row["src"], row["dst"])).distinct()

    # Create reverse edges to make the graph undirected
    reverse_edges = edges.map(lambda edge: (edge[1], edge[0]))
    all_edges = edges.union(reverse_edges).distinct()

    # Extract all unique airports from the edges
    airports = all_edges.flatMap(lambda edge: edge).distinct()

    # Assign unique IDs to each airport
    airport_id = airports.zipWithIndex().map(lambda x: (x[0], x[1]))
    id_to_airport = airport_id.map(lambda x: (x[1], x[0])).collectAsMap()
    airport_id_dict = airport_id.collectAsMap()

    # Broadcast the ID mappings for efficient access across the cluster
    broadcast_id_to_airport = flight_network_df.sparkSession.sparkContext.broadcast(id_to_airport)
    broadcast_airport_id = flight_network_df.sparkSession.sparkContext.broadcast(airport_id_dict)

    # Map edges using the integer IDs
    all_edges_id = all_edges.map(lambda x: (broadcast_airport_id.value[x[0]], broadcast_airport_id.value[x[1]])).distinct()

    # Build the graph as an RDD of (airport, list of neighbors)
    graph = all_edges_id.groupByKey().mapValues(list).persist(StorageLevel.MEMORY_AND_DISK)

    start_time = time.time()

    # Initialize labels: each airport's label is its own ID initially
    labels = all_edges_id.flatMap(lambda x: [x[0], x[1]]).distinct().map(lambda airport: (airport, airport)).persist(StorageLevel.MEMORY_AND_DISK)

    # Iteratively propagate labels until convergence or maximum iterations are reached
    current_labels = labels
    prev_labels = None
    iteration = 0
    MAX_ITERATIONS = 100  # Limit to prevent infinite loops

    while iteration < MAX_ITERATIONS:
        iteration += 1
        print(f"Iteration {iteration}")

        # Propagate labels
        new_labels = propagate_labels(current_labels, graph).persist(StorageLevel.MEMORY_AND_DISK)

        # Check for convergence by comparing with previous labels
        if prev_labels:
            differences = current_labels.join(new_labels) \
                .filter(lambda x: x[1][0] != x[1][1]) \
                .count()
            print(f"Differences from the previous iteration: {differences}")
            if differences == 0:
                print("Convergence achieved.")
                break
        else:
            print("First iteration, no comparison.")

        # Unpersist previous labels to free up memory
        if prev_labels:
            prev_labels.unpersist()

        # Update labels for the next iteration
        prev_labels = current_labels
        current_labels = new_labels

    # Warn if maximum iterations were reached without convergence
    if iteration == MAX_ITERATIONS:
        print("Warning: Maximum number of iterations reached without convergence.")

    end_time = time.time()
    execution_time = end_time - start_time

    # Collect the final labels
    final_labels = current_labels

    # Count the number of distinct connected components
    num_connected_components = final_labels.map(lambda x: x[1]).distinct().count()

    # Count the number of airports in each connected component
    component_sizes = final_labels.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).collect()

    # Sort the component sizes in descending order
    component_sizes_sorted = sorted(component_sizes, key=lambda x: x[1], reverse=True)

    # Identify the largest connected component
    if component_sizes_sorted:
        largest_component_label = component_sizes_sorted[0][0]
        largest_component_size = component_sizes_sorted[0][1]

        # Filter airports belonging to the largest connected component
        largest_component_airports = final_labels.filter(lambda x: x[1] == largest_component_label) \
            .map(lambda x: broadcast_id_to_airport.value[x[0]]).collect()
    else:
        largest_component_label = None
        largest_component_size = 0
        largest_component_airports = []

    return num_connected_components, component_sizes_sorted, largest_component_airports, execution_time


We execute the connected components analysis on the flight network dataset for the specified date range. The steps include:

1. **Defining the Analysis Period:**
   - Setting the start and end dates for the analysis.

2. **Data Preparation:**
   - Converting the `'Fly_date'` column to a proper date format to ensure accurate filtering.

3. **Identifying Connected Components:**
   - Utilizing the `identify_connected_components_rdd` function to determine the connected components within the defined date range.

4. **Displaying the Results:**
   - Printing the total number of connected components.
   - Listing the sizes of each connected component in descending order.
   - Enumerating the airports that belong to the largest connected component.
   - Showing the execution time of the analysis.


In [12]:
# Define the start and end dates for the analysis period
start_date = "2000-01-01"
end_date = "2000-05-31"

# Convert the 'Fly_date' column to a date type with the specified format
#flight_network_df = flight_network_df.withColumn("Fly_date", to_date(col("Fly_date"), "yyyy-MM-dd"))

# Identify connected components within the specified date range using the previously defined function
num_connected_components, component_sizes_sorted, largest_component_airports, execution_time = identify_connected_components_rdd(flight_network_df, start_date, end_date)

Iteration 1
First iteration, no comparison.
Iteration 2
Differences from the previous iteration: 208
Iteration 3
Differences from the previous iteration: 16
Iteration 4
Differences from the previous iteration: 0
Convergence achieved.


In [15]:
# Print the results of the connected components analysis
print("\n===== Connected Components Analysis =====\n")

# Print the number of connected components found within the specified date range
print(f"Number of connected components between {start_date} and {end_date}: {num_connected_components}\n")

# Print the sizes of each connected component, sorted in descending order
print("Sizes of each connected component (sorted in descending order):")
for idx, (component, size) in enumerate(component_sizes_sorted, start=1):
    print(f"{idx}. Component {component}: {size} airports")
print()

# Print the list of airports that belong to the largest connected component
print("Airports in the largest connected component:")
print(", ".join(f"{airport}" for airport in largest_component_airports))

# Print the execution time of the connected components analysis
print(f"Execution time: {execution_time}\n")


===== Connected Components Analysis =====

Number of connected components between 2000-01-01 and 2000-05-31: 2

Sizes of each connected component (sorted in descending order):
1. Component 0: 298 airports
2. Component 227: 2 airports

Airports in the largest connected component:
IND, JAX, SEA, FOE, JBR, RDU, TYS, EKO, EAT, CWA, IAD, MCO, MSN, SAV, FOD, PIT, BDL, ATL, FSM, DEC, JFK, DTW, BIL, ROC, DRT, ELP, YIP, SYR, JAC, SBP, MSP, BOS, BTR, LNK, DRO, SBA, BTV, SFB, MLU, EYW, TPA, ACT, BRD, ITH, UIN, SAT, CAK, STC, BIS, GUP, FAR, TUS, LCK, LCH, MKL, SHV, JNU, SFO, PWM, ICT, HNL, FLL, BMI, ALO, YUM, LAX, EWR, KTN, OGG, BRL, IAH, MEM, AVL, DHN, MRC, ORD, HLN, CRW, LBL, PAH, LAS, CAE, AZO, CHA, BVX, PBI, GEG, LIT, RST, BYH, ACY, OKC, DFW, MOB, ATY, GPT, RNO, YNG, PIR, CGI, MSY, SLC, SPI, TYR, SLN, GRR, AGS, OAK, EAU, APN, RDD, BLI, FAY, BJI, MMI, GSO, CMH, SMF, GJT, LRD, SAN, MBS, LBB, OGD, PNS, CLT, STL, MAF, TLH, ITO, BGR, RIC, BTM, ABQ, FLO, DCA, CLE, DET, LEX, CPR, SJC, MYR, FNT, ELM,

### 2. Compare the execution time and the results of your implementation with those of the GraphFrames package for identifying connected components. If there is any difference in the results, provide an explanation for why that might occur.


We create a new Spark session configured with GraphFrames to perform connected components analysis on the flight network dataset. This setup enables us to leverage graph-based processing capabilities for a more efficient and comprehensive analysis.

In [16]:
# Obtain the active SparkContext instance
sc = SparkContext.getOrCreate()

# Stop the SparkContext if you wish to create a new one
sc.stop()

# Initialize Spark Session with configurations
print("Initializing Spark Session...")

spark = SparkSession.builder \
        .appName("GraphFramesExample") \
        .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.0-s_2.12") \
        .config("spark.driver.memory", "15g") \
        .config("spark.executor.memory", "15g") \
        .config("spark.sql.shuffle.partitions", "100") \
        .getOrCreate()

print("... Spark Session Created")
sc = spark.sparkContext

Initializing Spark Session...
... Spark Session Created


#### Loading and Preparing the Flight Network Dataset

In [17]:
# Load the dataset as a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Airports2.csv')
flight_network_df = spark.createDataFrame(df)

# Display the schema to confirm
flight_network_df.printSchema()

root
 |-- Origin_airport: string (nullable = true)
 |-- Destination_airport: string (nullable = true)
 |-- Origin_city: string (nullable = true)
 |-- Destination_city: string (nullable = true)
 |-- Passengers: long (nullable = true)
 |-- Seats: long (nullable = true)
 |-- Flights: long (nullable = true)
 |-- Distance: long (nullable = true)
 |-- Fly_date: string (nullable = true)
 |-- Origin_population: long (nullable = true)
 |-- Destination_population: long (nullable = true)
 |-- Org_airport_lat: double (nullable = true)
 |-- Org_airport_long: double (nullable = true)
 |-- Dest_airport_lat: double (nullable = true)
 |-- Dest_airport_long: double (nullable = true)



We define the `identify_connected_components_graphframes` function, which leverages GraphFrames to identify connected components within the flight network dataset over a specified date range. This function filters the relevant flight data, constructs a graph of airports, computes the connected components, and returns key metrics such as the number of connected components, their sizes, the airports in the largest connected component, and the execution time of the analysis.


In [18]:
from graphframes import GraphFrame

def identify_connected_components_graphframes(flight_network_df, start_date, end_date):
    """
    Identifies the connected components in the flight network between start_date and end_date using GraphFrames.

    Parameters:
    - flight_network_df (DataFrame): Spark DataFrame containing flight data with columns
                                     'Origin_airport', 'Destination_airport', 'Fly_date'.
    - start_date (str): Start date of the period in 'YYYY-MM-DD' format.
    - end_date (str): End date of the period in 'YYYY-MM-DD' format.

    Returns:
    - num_connected_components (int): Number of connected components.
    - component_sizes_sorted (list of tuples): List of component sizes sorted in descending order.
    - largest_component_airports (list): List of airports in the largest connected component.
    - execution_time (float): Execution time in seconds.
    """

    # Filter flights within the specified date range and drop rows with null airports
    filtered_df = flight_network_df.filter(
        (col("Fly_date") >= start_date) & (col("Fly_date") <= end_date)
    ).na.drop(subset=["Origin_airport", "Destination_airport"])

    # Prepare vertices by selecting unique origin and destination airports
    vertices = filtered_df.select(col("Origin_airport").alias("id")).union(
        filtered_df.select(col("Destination_airport").alias("id"))
    ).distinct()

    # Prepare edges by selecting unique pairs of origin and destination airports
    edges = filtered_df.select(
        col("Origin_airport").alias("src"),
        col("Destination_airport").alias("dst")
    ).distinct()

    # Create the GraphFrame using the vertices and edges
    g = GraphFrame(vertices, edges)

    # Start the timer to measure execution time
    start_time = time.time()

    # Compute the connected components using GraphFrame's connectedComponents method
    result_graphframes = g.connectedComponents()

    # Stop the timer and calculate the execution time
    end_time = time.time()
    execution_time = end_time - start_time

    # Count the number of distinct connected components
    num_components = result_graphframes.select("component").distinct().count()

    # Count the number of airports in each connected component and sort them in descending order
    component_sizes = result_graphframes.groupBy("component").count().orderBy("count", ascending=False).collect()
    component_sizes_sorted = [(row["component"], row["count"]) for row in component_sizes]

    # Identify the largest connected component based on the sorted component sizes
    if component_sizes_sorted:
        largest_component_label = component_sizes_sorted[0][0]
        largest_component_size = component_sizes_sorted[0][1]

        # Filter airports that belong to the largest connected component
        largest_component_airports = result_graphframes.filter(
            col("component") == largest_component_label
        ).select("id").rdd.map(lambda row: row["id"]).collect()
    else:
        largest_component_label = None
        largest_component_size = 0
        largest_component_airports = []

    return num_components, component_sizes_sorted, largest_component_airports, execution_time

We execute the connected components analysis on the flight network dataset for the specified date range. The steps include:

1. **Defining the Analysis Period:**
   - Setting the start and end dates for the analysis.

2. **Data Preparation:**
   - Converting the `'Fly_date'` column to a proper date format to ensure accurate filtering.

3. **Identifying Connected Components:**
   - Utilizing the `identify_connected_components_graphframes` function to determine the connected components within the defined date range.

4. **Displaying the Results:**
   - Printing the total number of connected components.
   - Listing the sizes of each connected component in descending order.
   - Enumerating the airports that belong to the largest connected component.
   - Showing the execution time of the analysis.


In [19]:
# Define the date range for analysis
start_date = "2000-01-01"  # The start date for filtering flight data
end_date = "2000-05-31"    # The end date for filtering flight data

# Call the function to identify connected components within the specified date range using graphframes
num_connected_components, component_sizes_sorted, largest_component_airports, execution_time = identify_connected_components_graphframes(
    flight_network_df, start_date, end_date
)

# Print the analysis results

print("\n===== Connected Components Analysis =====\n")

# Display the total number of connected components identified within the specified date range
print(f"Number of connected components between {start_date} and {end_date}: {num_connected_components}\n")

# Print the sizes of each connected component, sorted from largest to smallest
print("Sizes of each connected component (sorted in descending order):")
for idx, (component, size) in enumerate(component_sizes_sorted, start=1):
    print(f"{idx}. Component {component}: {size} airports")
print()

# Print the list of airports that belong to the largest connected component
print("Airports in the largest connected component:")
print(", ".join(f"{airport}" for airport in largest_component_airports))

# Display the total time taken to perform the connected components analysis
print(f"Execution time: {execution_time} seconds\n")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling o1905.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallComman

Unfortunately, the analysis using GraphFrames cannot be completed because the graphframes package was not successfully imported. Despite configuring the Spark session with the appropriate package

```python
.config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.4-s_2.12")
```
the package fails to initialize properly, likely due to compatibility issues, unresolved dependencies and limitations of the runtime environment. As a result, we are unable to perform the requested analysis to identify the connected components of the flight network using GraphFrames.


## Algorithmic Question (AQ)

In [ ]:
import importlib, functions
importlib.reload(functions)

<module 'functions' from 'c:\\Users\\marta\\OneDrive\\Documenti\\GitHub\\ADM_HW5\\functions.py'>

### a) Write a pseudocode that describes the algorithm to find the cheapest route with at most k stops.

---

**Function**: $createAdjacencyList(flights, n)$

**Input**:
- $flights$: A list of edges in the form $[u, v, \text{cost}]$, where:
  - $u\in V$: The *source vertex*;
  - $v\in V$: The *destination vertex*;
  - $cost$: The *weight of the edge* from $u$ to $v$.
- $n$: The *total number of vertices* in the graph.

**Output**: An *adjacency list* where each vertex $u$ is associated with a list of lists $[v, cost]$.

&nbsp;&nbsp;&nbsp;$adj\_list$ &larr; **initialize** list of empty lists of size $n$

&nbsp;&nbsp;&nbsp;**for each** flight $[u, v, cost]$ **in** $flights$ **do**\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**add** $[v, cost]$ **to** $adj\_list[u]$

&nbsp;&nbsp;&nbsp;**return** $adj\_list$


---

**Function**: $minCostWithMaxKEdgesDFS(G, currentNode, destination, maxStops, currentCost, minCostSoFar, visitedNodes)$

**Input**:
- $G = (V, E)$: Graph in *adjacency-list representation*;
- $currentNode \in V$: The *current vertex* being explored;
- $destination \in V$: The *destination vertex*;
- $maxStops$: The *maximum number of stops allowed*;
- $currentCost$: The *cumulative cost of the path so far*;
- $minCostSoFar$: The *minimum cost found so far*;
- $visitedNodes$: The set of distinct nodes already visited in the current DFS path.

**Output**: The *updated minimum cost to travel from* $currentNode$ *to* $destination$ *within paths that include at most* $maxStops+1$ *nodes*.

&nbsp;&nbsp;&nbsp;**if** $currentNode == destination$ **then**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**return** $\mathbf{min}(currentCost, minCostSoFar)$  

&nbsp;&nbsp;&nbsp;**if** $maxStops < 0$ **then**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**return** $minCostSoFar$  

&nbsp;&nbsp;&nbsp;**add** $currentNode$ to $visitedNodes$  

&nbsp;&nbsp;&nbsp;**for each** *list* $[neighbor, edgeCost]$ **in** $G[currentNode]$ **do**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**if** $neighbor \notin visitedNodes$ **and** $currentCost + edgeCost < minCostSoFar$ **then**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$updatedMinCost$ &larr; $\mathbf{minCostWithMaxKEdgesDFS}(G, neighbor, destination, maxStops - 1, currentCost + edgeCost, minCostSoFar, visitedNodes)$  

&nbsp;&nbsp;&nbsp;**remove** $currentNode$ from $visitedNodes$  

&nbsp;&nbsp;&nbsp;**return** $updatedMinCost$

---
**Function**: $findCheapestRouteDFS(n, flights, src, dest, k)$

**Input**:
- $n$: The *total number of vertices* in the graph.
- $flights$: A list of edges in the form $[u, v, cost]$, where:
  - $u$: The *source vertex*;
  - $v$: The *destination vertex*;
  - $cost$: *weight of the edge* from $u$ to $v$.
- $src \in V$: The *source vertex*;
- $dst\in V$: The *destination vertex*;
- $k$: The *maximum number of edges allowed*.

**Output**: The *minimum cost* to travel from $src$ to $dst$ crossing at the most $k$ edges, or $-1$ if no valid path exists.

&nbsp;&nbsp;&nbsp;$G$ &larr; $\mathbf{createAdjacencyList}(flights, n)$

&nbsp;&nbsp;&nbsp;$min\_cost$ &larr; $\mathbf{minCostWithMaxKEdgesDFS}(G, src, dst, k, 0, \infty, \emptyset)$

&nbsp;&nbsp;&nbsp;**if** $min\_cost < \infty$ **then**\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**return** $min\_cost$

&nbsp;&nbsp;&nbsp;**return** $-1$

#### Correctness analysis

The algorithm systematically explores all possible simple paths from the source to the destination whose length does not exceed $k+1$ nodes. First, the graph is transformed into an adjacency list via $\mathbf{createAdjacencyList}$, ensuring that for every node, we know all its outgoing edges and the corresponding costs. During the depth-first search in $\mathbf{minCostWithMaxKEdgesDFS}$, each recursive call only proceeds if the number of remaining stops is non-negative, thereby enforcing the constraint that no path longer than $k+1$ nodes (i.e., $k$ edges) is explored. Whenever the current node matches the destination, the minimum cost so far is updated if the current path’s cost is lower, so the global minimum for paths of acceptable length is eventually discovered. In addition, the algorithm temporarily includes the current node in the set of visited nodes to prevent revisiting the same node within one path, then removes it upon returning from the recursion, thereby considering only simple paths. This ensures no cycles inflate the cost or cause infinite recursion. Hence, by the time $\mathbf{minCostWithMaxKEdgesDFS}$ completes, it has exhaustively checked all valid simple paths of length at most $k+1$ nodes and recorded the cheapest among them; if no such path exists, the function returns $\infty$, causing the top-level $\mathbf{findCheapestRouteDFS}$ to return $-1$. Thus, the algorithm correctly identifies either the minimal cost or the impossibility of reaching the destination under the specified constraints.


### b) Implement the algorithm in Python and simulate the given test cases.

We have implemented the `find_cheapest_route_DFS` main function, along with its helper routines `create_adjacency_list` and `min_cost_with_max_K_edges_DFS`, in the `functions.py` module. For a more in-depth explanation of how these functions work, please refer to the detailed comments included there. Below, we call `find_cheapest_route_DFS` on the provided test cases and verify that it returns the expected results.

In [ ]:
from functions import find_cheapest_route_DFS

n = [4, 3, 3, 4, 4]
flights = [[[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]],
           [[0, 1, 100], [1, 2, 100], [0, 2, 500]],
           [[0, 1, 100], [1, 2, 100], [0, 2, 500]],
           [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 300]],
           [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 200]]]
src = [0, 0, 0, 0, 0]
dst = [3, 2, 2, 3, 3]
k = [1, 1, 0, 2, 2]

for i in range(len(n)):
    result = find_cheapest_route_DFS(n[i], flights[i], src[i], dst[i], k[i])
    print("Flights: " + str(flights[i]) +
          ", source: " + str(src[i]) +
          ", destination: " + str(dst[i]) +
          ", maximum number of stops: " + str(k[i]) +
          "\nCheapest route: " + str(result) + "\n")

Flights: [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]], source: 0, destination: 3, maximum number of stops: 1
Cheapest route: 700

Flights: [[0, 1, 100], [1, 2, 100], [0, 2, 500]], source: 0, destination: 2, maximum number of stops: 1
Cheapest route: 200

Flights: [[0, 1, 100], [1, 2, 100], [0, 2, 500]], source: 0, destination: 2, maximum number of stops: 0
Cheapest route: 500

Flights: [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 300]], source: 0, destination: 3, maximum number of stops: 2
Cheapest route: 400

Flights: [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 200]], source: 0, destination: 3, maximum number of stops: 2
Cheapest route: 400



### c) Analyze the algorithm's efficiency. Provide its time complexity and space complexity, and explain whether it is efficient for large graphs (e.g., n > 100).

#### Time Complexity Analysis
The main algorithm is $\mathbf{findCheapestRouteDFS}$, whose complexity depends on the algorithms $\mathbf{createAdjacencyList}$ and $\mathbf{minCostWithMaxKEdgesDFS}$. Let us analyze their contributions in detail.

The algorithm $\mathbf{createAdjacencyList}$ constructs the graph representation using an adjacency list. It initializes an empty list for each of the $n$ nodes, which takes $O(n)$, and iterates over all $m$ edges to populate the adjacency list, requiring $O(m)$. Therefore, the complexity of $\mathbf{createAdjacencyList}$ is $O(m + n)$.

The computationally intensive part is the algorithm $\mathbf{minCostWithMaxKEdgesDFS}$, which performs a depth-first search (DFS) with a constraint on the maximum number of stops $k$. The algorithm explores all possible simple paths (i.e., crossing each node in the path exactly once) starting from $src$, and the maximum depth of the recursion is $k+1$ (corresponding to $k$ stops). Importantly, the algorithm does not only visit paths of length $k+1$ but also considers paths of smaller lengths when the condition $src == dst$ is satisfied before reaching the $k$-stop limit.

To analyze its complexity, let us determine the number of recursive calls. At each level of recursion, the DFS explores all neighbors of the current node. Let $G=(V,E)$ be the graph built by $\mathbf{createAdjacencyList}$, where $n=|V|$ and $m=|E|$. In the worst case, the graph is complete, meaning there exists an edge between each pair of distinct nodes, resulting in $deg(u)=n-1 \; \forall u \in V$ and $m = \dfrac{n(n-1)}{2}$.

Moreover, in the worst case, the algorithm analyzes all possible simple paths of length $k+1$ starting from the source node, with the condition $src == dst$ being satisfied only when the stops finish or never. Let us analyze the time complexity in the worst case. The first call inserts $src$ into $visitedNodes$ and generates $n-1$ calls. Each of these generates $n-2$ calls at the next level, which in turn generate $n-3$ calls, and so on, until the $(k+1)^{th}$ level. At this depth, the algorithm generates the final recursive calls, with the branching factor reduced to $n-k-1$. This occurs because $visitedNodes$ at each call contains the nodes previously visited for the specific path, avoiding revisits and ensuring the algorithm explores only simple paths of length at most $k+1$. When a recursive call ends, it removes the specific $src$ from $visitedNodes$, allowing backtracking. Thus, the number of recursive calls $T(n, k)$ can be expressed as:

\begin{align*}
T(n,k) &= (n-1) + (n-1)(n-2) + (n-1)(n-2)(n-3) + \dots + (n-1)(n-2)\cdots(n-k)(n-k-1) = \sum_{i=1}^{k+1} \prod_{j=0}^{i-1} (n-1-j)
\end{align*}

To simplify, we estimate an upper bound:
\begin{equation*}
T(n,k) \leq \sum_{i=1}^{k+1} (n-1)^i.
\end{equation*}

The summation represents a geometric progression with ratio $(n-1)$. The closed form for this summation is:
\begin{align*}
T(n,k) &\leq \dfrac{(n-1)^{k+2} - (n-1)}{n-2} \approx (n-1)^{k+1}
\end{align*}

This simplifies asymptotically to:
\begin{equation*}
T(n,k) \in O((n-1)^{k+1}).
\end{equation*}

Thus, the overall time complexity of $\mathbf{findCheapestRouteDFS}$ is $O(m + (n-1)^{k+1})$, where $m$ is the number of edges and $(n-1)^{k+1}$ is the dominating term in the worst-case scenario.

#### Space Compelxity Analysis
When analyzing the space complexity of $\mathbf{findCheapestRouteDFS}$, we take into account the adjacency list used to represent the graph, the recursive call stack, and the auxiliary data structures required during execution. The graph itself is stored using an adjacency list that requires $O(n + m)$ space.

Beyond the space needed to store the input graph, the algorithm uses additional memory for the recursion and for tracking visited nodes in the current path. The recursive call stack can grow up to a depth of $k+1$ calls, where $k$ is the maximum number of stops allowed. Consequently, the worst-case space usage for the call stack is $O(k)$. In parallel, the set of visited nodes holds at most $k+1$ distinct vertices at any point, adding another $O(k)$ to the total auxiliary space.

Combining these factors, we see that the total space consumption can be expressed as $O(k) + O(k) + O(n + m)$, which simplifies to $O(k + n + m)$. Since in the worst case $k$ can be bounded by $n$, this further reduces to $O(n + m)$. Therefore, if we consider only the additional space used during the DFS (beyond the input), it is $O(k)$. However, when we include the adjacency list needed to store the graph, the overall space complexity is $O(n + m)$.


#### Efficiency for large graphs
The efficiency of $\mathbf{findCheapestRouteDFS}$ for large graphs is significantly influenced by the structure of the graph and the parameter $k$, which limits the maximum number of stops. While the adjacency list construction is efficient, taking $O(m + n)$, the depth-first search component introduces exponential growth due to the $(n-1)^{k+1}$ term in its time complexity.\
For sparse graphs, where the number of edges $m$ is close to $n$, and for small values of $k$, the algorithm can handle larger graphs effectively because the exponential term $(n-1)^{k+1}$ remains manageable. However, as $k$ increases, or if the graph is dense ($m \approx n^2$), the number of paths the DFS must explore grows rapidly, making the algorithm impractical for large graphs.\
In summary, while the algorithm is suitable for small $k$ and sparse graphs, it is not efficient for large graphs with high connectivity or large $k$, as the exponential growth dominates the computational cost.

### d) Optimize the algorithm to handle larger graphs. Provide an updated pseudocode and analyze the computational complexity of your optimization.
---
**Function**: $minCostWithMaxKEdgesPol(G, src, dst, k)$

**Input**:
- $G = (V, E)$: Graph in *adjacency-list representation*;
- $src \in V$: The *source vertex*;
- $dst \in V$: The *destination vertex*;
- $k$: The *maximum number of edges allowed*.

**Output**: The *minimum cost to travel from* $src$ *to* $dst$ *using at most* $k+1$ *edges, or* $\infty$ *if no valid path exists*.

&nbsp;&nbsp;&nbsp; $numNodes$ &larr; $\mathbf{len}(V)$  
&nbsp;&nbsp;&nbsp; $INF$ &larr; $\infty$  

&nbsp;&nbsp;&nbsp; **Initialize** $minCostPerEdge[i][v]$ for all $i \in [0, k+1]$ and $v \in V$:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $minCostPerEdge[i][v]$ &larr; $INF$  

&nbsp;&nbsp;&nbsp;$minCostPerEdge[0][src]$ &larr; $0$  

&nbsp;&nbsp;&nbsp; **for each** $edgeCount \in [1, k+1]$ **do**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $minCostPerEdge[edgeCount][.]$ &larr; $minCostPerEdge[edgeCount-1][.]$  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **for each** vertex $u \in V$:\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **if** $minCostPerEdge[edgeCount-1][u] \neq \infty$ **then**:  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **for each** $(v, weight) \in G[u]$ **do**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$newCost$ &larr; $minCostPerEdge[edgeCount-1][u] + weight$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **if** $newCost < minCostPerEdge[edgeCount][v]$ **then**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$minCostPerEdge[edgeCount][v]$ &larr; $newCost$  

&nbsp;&nbsp;&nbsp; **return** $\mathbf{min}(minCostPerEdge[edgeCount][dst] \, \forall \, edgeCount \in [0, k+1])$

---
**Function**: $findCheapestRoutePol(n, flights, src, dest, k)$

**Input**:
- $n$: The *total number of vertices* in the graph.
- $flights$: A list of edges in the form $[u, v, cost]$, where:
  - $u$: The *source vertex*;
  - $v$: The *destination vertex*;
  - $cost$: *weight of the edge* from $u$ to $v$.
- $src \in V$: The *source vertex*;
- $dst\in V$: The *destination vertex*;
- $k$: The *maximum number of edges allowed*.

**Output**: The *minimum cost* to travel from $src$ to $dst$ crossing at the most $k$ edges, or $-1$ if no valid path exists.

&nbsp;&nbsp;&nbsp; $G$ &larr; $\mathbf{createAdjacencyList}(flights, n)$

&nbsp;&nbsp;&nbsp; $min\_cost$ &larr; $\mathbf{minCostWithMaxKEdgesPol}(G, src, dst, k)$

&nbsp;&nbsp;&nbsp; **if** $min\_cost < \infty$ **then**\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **return** $min\_cost$

&nbsp;&nbsp;&nbsp; **return** $-1$

---

#### Correctness Analysis
This approach relies on a dynamic-programming-like strategy to compute the minimum cost of reaching each vertex using up to $k+1$ edges. The array $minCostPerEdge[i][v]$ stores the minimal cost to reach vertex $v$ with exactly $i$ edges, and it is initialized to infinity for all $i$ and $v$, except for $minCostPerEdge[0][src]$ which is set to $0$. On each iteration from $1$ to $k+1$, the values of $minCostPerEdge[i]$ are first copied from $minCostPerEdge[i-1]$, which ensures that no vertex’s cost worsens from one step to the next. The algorithm then looks at each vertex $u$ for which $minCostPerEdge[i-1][u]$ is not infinity, and examines every edge $(u \rightarrow v)$ with weight $weight$. If traveling from $src$ to $u$ with $i-1$ edges costs $minCostPerEdge[i-1][u]$ and then continuing to $v$ adds $weight$, the new total cost $newCost$ is compared to the current $minCostPerEdge[i][v]$. If $newCost$ is smaller, $minCostPerEdge[i][v]$ is updated. By the end of these $k+1$ rounds, $minCostPerEdge[i][v]$ will hold the lowest possible cost from $src$ to $v$ with at most $i$ edges. The minimum across all $i \in [0, k+1]$ for the destination $dst$ thus reflects the true minimal cost to get from $src$ to $dst$ with at most $k+1$ edges. If no cost was ever improved from infinity, then it follows that no valid path with at most $k$ edges (or $k+1$ nodes) exists, and $\infty$ is returned. Hence the algorithm correctly finds the cheapest cost under the given edge constraint, or deduces that no such route is possible.


#### Time Complexity Analysis
The function $\mathbf{createAdjacencyList(flights, n)}$ is the same as previously discussed, used for building the adjacency list representation of the graph. Its time complexity is $O(m+n)$, where $m$ is the number of edges and $n$ is the number of nodes. The overall time complexity of the $\mathbf{findCheapestRoutePol}$ algorithm is determined by the combination of $\mathbf{createAdjacencyList}$ and $\mathbf{minCostWithMaxKEdgesPol}$.

Analyzing the $\mathbf{minCostWithMaxKEdgesPol}$ function in detail, we first compute the number of nodes, which takes $O(n)$. Then, initializing the matrix $minCostPerEdge$ to represent the minimum cost for each node and edge count requires $O(n \cdot k)$, since the matrix has dimensions $(k+2) \times n$. Setting $minCostPerEdge[0][src]$ to zero is a single operation and takes $O(1)$.

The primary computational effort lies in the nested loops. The outer loop iterates over the number of edge counts, from $1$ to $k+1$, resulting in $O(k)$ iterations. For each iteration, the previous row of the matrix is copied to the current row, which takes $O(n)$. Inside this loop, we iterate over all nodes, amounting to $O(n)$ operations for the middle loop. For each node, we then iterate over its neighbors, and in the worst case, where the graph is complete, each node has up to $n-1$ neighbors. Thus, the inner-most loop processes all neighbors and, when combined with the middle loop, results in $O(n^2)$ operations for each iteration of the outer loop.

Given that the outer loop runs $O(k)$ times, the total time complexity of the nested loops can be expressed as:
\begin{align*}
T(n, k) = O(k) \cdot O(n^2) = O(k \cdot n^2).
\end{align*}

After the nested loops complete, the algorithm computes the minimum cost among all valid edge counts from $0$ to $k+1$. This requires iterating over $k+2$ values and takes $O(k)$.

Combining all these components, the total time complexity of $\mathbf{minCostWithMaxKEdgesPol}$ is $O(k \cdot n^2)$. As $O(k \cdot n^2)$ dominates, it determines the overall time complexity of the $\mathbf{findCheapestRoutePol}$ algorithm, exceeding the $O(m+n)$ complexity of creating the adjacency list.

#### Space Complexity Analysis
The space complexity of the $\mathbf{minCostWithMaxKEdgesPol}$ algorithm is primarily determined by the $minCostPerEdge$ matrix, which has dimensions $(k+2) \times n$. Thus, the space complexity is $O(k \cdot n)$. No additional significant memory is used apart from a few variables for computation, making the space usage manageable for most practical cases. Considering the memory required to store the graph's adjacency list representation, the total space complexity of $\mathbf{findCheapestRoutePol}$ is $O(k \cdot n + m)$.

#### Efficiency for large graphs
The $\mathbf{minCostWithMaxKEdgesPol}$ algorithm is significantly more efficient in terms of time complexity compared to the DFS-based algorithm used previously. The DFS-based approach explores all possible paths with up to $k+1$ nodes, leading to an exponential time complexity of $O((n-1)^{k+1})$ in the worst case, as it examines a vast number of potential combinations of nodes. In contrast, $\mathbf{minCostWithMaxKEdgesPol}$ employs a dynamic programming approach with a time complexity of $O(k \cdot n^2)$ or equivalently $O(k \cdot (n + m))$. This improvement is substantial, particularly for large graphs where $n$ and $m$ are significant. The quadratic dependency on $n$ ensures that the algorithm scales better, even for large $k$, making it far more practical for real-world applications.

In terms of memory usage, the DFS-based algorithm is more efficient. Disregarding the memory required for the graph's representation, the DFS approach requires $O(k)$ space for the recursive stack and the visited set. On the other hand, $\mathbf{minCostWithMaxKEdgesPol}$ uses $O(k \cdot n)$ space to store the dynamic programming matrix $minCostPerEdge$, which grows linearly with both $k$ and $n$. While this increased memory usage might appear to be a disadvantage, the trade-off is worthwhile in this context.

In computational problems like these, reducing the time complexity is often far more critical than minimizing memory usage. A faster algorithm enables the solution of larger and more complex instances within a reasonable time frame, which is essential for practical applications. While memory efficiency is important, modern computational systems generally have sufficient memory to handle the $O(k \cdot n)$ requirement of this algorithm, especially when $k$ and $n$ are not excessively large.

Therefore, the $\mathbf{minCostWithMaxKEdgesPol}$ algorithm strikes a better balance between time and space efficiency, making it a superior choice for solving this problem, especially for large graphs or higher values of $k$. Sacrificing a modest increase in memory for a dramatic improvement in runtime is the more practical trade-off in this scenario.

#### Python Implementation
We have implemented the `find_cheapest_route_pol` function, which invokes `create_adjacency_list` and the dynamic-programming-based routine `min_cost_with_max_K_edges_pol`, in the `functions.py` module. We used the same test cases as before, and the function produced the same correct results, confirming its validity. For a more detailed explanation, please refer to `functions.py`, where the code is enriched with extensive comments.

In [ ]:
from functions import find_cheapest_route_pol

n = [5, 3, 3, 4, 4]
flights = [[[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]],
           [[0, 1, 100], [1, 2, 100], [0, 2, 500]],
           [[0, 1, 100], [1, 2, 100], [0, 2, 500]],
           [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 300]],
           [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 200]]]
src = [0, 0, 0, 0, 0]
dst = [3, 2, 2, 3, 3]
k = [1, 1, 0, 2, 2]

for i in range(len(n)):
    result = find_cheapest_route_pol(n[i], flights[i], src[i], dst[i], k[i])
    print("Flights: " + str(flights[i]) +
          ", source: " + str(src[i]) +
          ", destination: " + str(dst[i]) +
          ", maximum number of stops: " + str(k[i]) +
          "\nCheapest route: " + str(result) + "\n")

Flights: [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]], source: 0, destination: 3, maximum number of stops: 1
Cheapest route: 700

Flights: [[0, 1, 100], [1, 2, 100], [0, 2, 500]], source: 0, destination: 2, maximum number of stops: 1
Cheapest route: 200

Flights: [[0, 1, 100], [1, 2, 100], [0, 2, 500]], source: 0, destination: 2, maximum number of stops: 0
Cheapest route: 500

Flights: [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 300]], source: 0, destination: 3, maximum number of stops: 2
Cheapest route: 400

Flights: [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 200]], source: 0, destination: 3, maximum number of stops: 2
Cheapest route: 400



### e) Ask LLM (e.g., ChatGPT) for an optimized version of your algorithm. Compare its solution to yours in terms of performance, time complexity, and correctness.

We requested ChatGPT to provide an optimized version of the $\mathbf{minCostWithMaxKEdgesPol}$ algorithm, and it returned a solution that adapts Dijkstra’s approach by restricting paths to at most $k+1$ flights. The graph is represented as an adjacency list, constructed using the same $\mathbf{createAdjacencyList}$ function referenced in the earlier pseudocode scripts. Below is the pseudocode that captures ChatGPT’s idea, employing a priority queue to find the cheapest route under the given stop constraint.

---
**Function**: $findMinCostDijkstra(G, src, dst, k)$

**Input**:
- $G = (V, E)$: Graph in *adjacency-list representation*;
- $src \in V$: The *source vertex*;
- $dst \in V$: The *destination vertex*;
- $k$: The *maximum number of stops allowed*.

**Output**: The *minimum cost to travel from* $src$ *to* $dst$ *within paths that include at most* $k+1$ *nodes*.

&nbsp;&nbsp;&nbsp;**for each** vertex $node \in V$:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$minDistanceK[node]$ &larr; $\infty$  

&nbsp;&nbsp;&nbsp; $minDistanceK[src]$ &larr; 0  
&nbsp;&nbsp;&nbsp; $priority\_queue$ &larr; $\emptyset$  
&nbsp;&nbsp;&nbsp; $priority\_queue.\mathbf{insert}((0, src, 0, \{src\}))$  

&nbsp;&nbsp;&nbsp;**while** \(**not** $priority\_queue.\mathbf{isEmpty()}$\) **do**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $current\_node, current\_cost, current\_stops, current\_visited$ &larr; $priority\_queue.\mathbf{deleteMin()}$  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**if** $current\_stops \leq k+1$ **then**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**if** $current\_node == dst$ **then**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$minDistanceK[dst]$ &larr; $current\_cost$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**break**  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**for each** *list* $[neighbor, edge\_cost]\in G[current\_node]$ **do**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**if** $neighbor \notin current\_visited$ **then**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$new\_cost$ &larr; $current\_cost + edge\_cost$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**if** $minDistanceK[neighbor] > new\_cost$ **or** $current\_stops + 1 < stops[neighbor]$ **then**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$minDistanceK[neighbor]$ &larr; $new\_cost$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$new\_visited$ &larr; $current\_visited \cup \{neighbor\}$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$priority\_queue.\mathbf{insert}((minDistanceK[neighbor], neighbor, current\_stops + 1, new\_visited))$  

&nbsp;&nbsp;&nbsp;**return** $minDistanceK[dst]$

---
**Function**: $findCheapestRouteDijkstra(n, flights, src, dest, k)$

**Input**:
- $n$: The *total number of vertices* in the graph.
- $flights$: A list of edges in the form $[u, v, cost]$, where:
  - $u$: The *source vertex*;
  - $v$: The *destination vertex*;
  - $cost$: *weight of the edge* from $u$ to $v$.
- $src \in V$: The *source vertex*;
- $dst\in V$: The *destination vertex*;
- $k$: The *maximum number of edges allowed*.

**Output**: The *minimum cost* to travel from $src$ to $dst$ crossing at the most $k$ edges, or $-1$ if no valid path exists.

&nbsp;&nbsp;&nbsp;$G$ &larr; $\mathbf{createAdjacencyList}(flights, n)$

&nbsp;&nbsp;&nbsp;$min\_cost$ &larr; $\mathbf{findMinCostDijkstra}(G, src, dst, k)$

&nbsp;&nbsp;&nbsp;**if** $min\_cost < \infty$ **then**\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**return** $min\_cost$

&nbsp;&nbsp;&nbsp;**return** $-1$

#### Correctness Analysis
The algorithm is correct because it processes states in increasing order of total cost, ensuring that when a state with the destination node is finally extracted from the priority queue, the corresponding cost is indeed minimal among all paths that use at most $k+1$ flights. It starts by pushing into the queue a single state with cost 0, node equal to the source, 0 stops, and a visited set containing only the source. By always extracting the state with the smallest cost first, it imitates Dijkstra’s logic, but it also keeps a set of visited nodes to forbid revisiting them in the same path and a stops counter that cannot exceed $k+1$. This means that each simple path of length up to $k+1$ flights is eventually considered in strictly increasing order of cost. If a state with node equal to the destination is popped, that state’s cost must be minimal among all valid paths; if such a state never appears, it implies that no valid path exists, and the algorithm returns -1.

#### Time Complexity Analysis
ChatGPT claimed that each node would appear in the priority queue at most $k+1$ times, leading to $O(n \cdot k)$ total entries and a cost of $O(\log(n \cdot k))$ per extract-min operation, which would imply a time complexity of $O(m \cdot \log(n \cdot k))$. However, this analysis overlooks the fact that the code stores the entire set of visited nodes in each state. This means that the same node can be inserted multiple times with different subsets of visited nodes, and these subsets can be numerous when $k$ is not very small. Indeed, each path can include up to $k+1$ distinct nodes, and a node can appear in the priority queue along with any of the subsets of already visited nodes. In the worst-case scenario, the number of such subsets can be on the order of $n^{k+1}$, as each of the $k+1$ nodes in the path can be chosen from $n$ possibilities, creating a combinatorial explosion. Once we acknowledge that the algorithm might enqueue on the order of $n^{k+1}$ states, the extract-min step on the priority queue, costing $O(\log(n^{k+1}))$, turns into
\begin{align*}
O\bigl(n^{k+1} \cdot \log(n^{k+1})\bigr),
\end{align*}
and since $\log(n^{k+1}) = (k+1)\log(n)$, this simplifies to
\begin{align*}
O\bigl(n^{k+1} \cdot k \log n\bigr).
\end{align*}
Therefore, the actual worst-case time complexity of the code is exponential in $k$, rather than the more modest $O(m \cdot \log(n \cdot k))$ described by ChatGPT. This exponential behavior stems from the large number of potential visited-subset configurations that can appear in the priority queue, invalidating ChatGPT’s claim that the total enqueues would be limited to just one state per node per stop-count.

#### Comparison with Our Dynamic Programming Approach

Our original algorithm employs a dynamic programming strategy, where we iteratively compute the minimum cost to reach each node with up to $k+1$ edges. The DP table is updated in each iteration based on the costs from the previous step, ensuring that we consider all possible paths within the stop constraint without revisiting nodes in the same path.

ChatGPT's Dijkstra-like approach was initially described as having a time complexity of $O(m \cdot \log(n \cdot k))$, where $m$ is the number of flights and $n$ is the number of nodes. This suggests that the algorithm scales efficiently with the number of flights and nodes, especially when $k$ is relatively small. However, this analysis does not account for the additional complexity introduced by tracking the set of visited nodes within each state. By maintaining these visited sets, the number of possible states that the algorithm must process can grow exponentially with respect to $k$. Specifically, in the worst case, the number of states can reach $O(n^{k+1})$, leading to a time complexity of $O(n^{k+1} \cdot k \cdot \log n)$. This exponential growth makes ChatGPT's approach impractical for larger values of $k$, as the computation time becomes prohibitively large. In contrast, our dynamic programming approach operates with a time complexity of $O(k \cdot n^2)$, which scales linearly with both the number of allowed stops and the number of flights. This linear relationship ensures that our algorithm remains efficient even as the size of the graph and the value of $k$ increase. By avoiding the need to track visited node sets and instead iteratively building up the minimum costs, our approach circumvents the combinatorial explosion inherent in ChatGPT's method.

Both ChatGPT's approach and our dynamic programming method correctly identify the minimum cost path within the specified number of stops. ChatGPT's algorithm ensures correctness by processing nodes in order of increasing cost and adhering to the stop constraint. However, the inefficiency introduced by tracking visited sets compromises its practicality for larger problems. Our dynamic programming approach guarantees correctness by systematically updating the minimum costs for each node across all permissible numbers of stops, ensuring that the optimal path is found without unnecessary computational overhead.

In conclusion, while ChatGPT's optimized Dijkstra-like algorithm maintains correctness in finding the cheapest route within the stop constraint, its exponential time complexity relative to $k$ renders it inefficient for larger values of $k$. Our dynamic programming approach, with its linear time complexity concerning both $k$ and the number of flights, offers a more scalable and practical solution. This makes our method better suited for handling extensive graphs and higher stop constraints, ensuring both efficiency and accuracy in diverse scenarios.

#### Python Implementation
We have implemented the `find_cheapest_route_Dijkstra` function in Python, which is located within the `functions.py` module. This function leverages the helper functions `create_adjacency_list` for constructing the graph's adjacency list and `min_cost_with_max_K_edges_Dijkstra` for computing the minimum cost route with the specified number of stops. To ensure the reliability and correctness of our implementation, we tested `find_cheapest_route_Dijkstra` using the same test cases previously described. The results consistently matched the expected outcomes, confirming that the algorithm performs as intended. For a more detailed explanation of the implementation and comprehensive comments, please refer to the `functions.py` file.

In [ ]:
from functions import find_cheapest_route_Dijkstra

n = [5, 3, 3, 4, 4]
flights = [[[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]],
           [[0, 1, 100], [1, 2, 100], [0, 2, 500]],
           [[0, 1, 100], [1, 2, 100], [0, 2, 500]],
           [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 300]],
           [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 200]]]
src = [0, 0, 0, 0, 0]
dst = [3, 2, 2, 3, 3]
k = [1, 1, 0, 2, 2]

for i in range(len(n)):
    result = find_cheapest_route_Dijkstra(n[i], flights[i], src[i], dst[i], k[i])
    print("Flights: " + str(flights[i]) +
          ", source: " + str(src[i]) +
          ", destination: " + str(dst[i]) +
          ", maximum number of stops: " + str(k[i]) +
          "\nCheapest route: " + str(result) + "\n")

Flights: [[0, 1, 100], [1, 2, 100], [2, 0, 100], [1, 3, 600], [2, 3, 200]], source: 0, destination: 3, maximum number of stops: 1
Cheapest route: 700

Flights: [[0, 1, 100], [1, 2, 100], [0, 2, 500]], source: 0, destination: 2, maximum number of stops: 1
Cheapest route: 200

Flights: [[0, 1, 100], [1, 2, 100], [0, 2, 500]], source: 0, destination: 2, maximum number of stops: 0
Cheapest route: 500

Flights: [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 300]], source: 0, destination: 3, maximum number of stops: 2
Cheapest route: 400

Flights: [[0, 1, 100], [0, 2, 200], [1, 3, 300], [2, 3, 200]], source: 0, destination: 3, maximum number of stops: 2
Cheapest route: 400

